In [1]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import socket

# Load the base overlay
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODA
#include "gpio.h"

// Function to set a GPIO pin as output and write a value
void write_gpio(unsigned int pin, unsigned int val) {
    if (val > 1) {
        return;
    }
    gpio pin_out = gpio_open(pin); // Open the GPIO pin
    gpio_set_direction(pin_out, GPIO_OUT); // Set direction as output
    gpio_write(pin_out, val); // Write the specified value
}

// Initialize GPIO pins for the buzzer
void init_gpio() {
    for (unsigned int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin); // Open the GPIO pin
        gpio_set_direction(pin_out, GPIO_OUT); // Set direction as output
    }
}

In [ ]:
btns = base.buttons  # Initialize buttons
def tone(freq, duration):
    """Generate a tone on the buzzer using PWM."""
    print(f"[INFO] Activating buzzer at {freq}Hz for {duration}s")
    buzzer.generate(freq, duration)  # Generate the tone
    time.sleep(duration)  # Let the tone play
    buzzer.stop()  # Stop buzzer after duration
    print("[INFO] Buzzer OFF")

# Function to generate a tone (buzzer simulation)
def tone(freq, duration):
    print(f"[INFO] Activating buzzer at {freq}Hz for {duration}s")
    # Assuming GPIO 0 is used for buzzer
    write_gpio(0, 1)  # Turn buzzer ON
    time.sleep(duration)
    write_gpio(0, 0)  # Turn buzzer OFF
    print("[INFO] Buzzer OFF")

# Server function
def server(server_ip, server_port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Allow port reuse
    sock.bind((server_ip, server_port))
    sock.listen(5)  # Allow multiple clients
    print(f"[INFO] Server listening on {server_ip}:{server_port}")

    while True:
        connection_socket, client_address = sock.accept()
        print(f"[INFO] Connection received from {client_address}")
        multiprocessing.Process(target=handle_client, args=(connection_socket,)).start()

# Handle client requests
def handle_client(connection_socket):
    try:
        message = connection_socket.recv(2048).decode()
        print(f"[INFO] Received message: {message}")  # ✅ Debugging print

        if message == 'TONE':
            print("[INFO] Playing tone...")  # ✅ Confirm TONE message
            tone(1000, 0.5)  # Play buzzer
        elif message == 'DISCONNECT':
            print("[INFO] Client disconnected.")

    finally:
        connection_socket.close()

if __name__ == "__main__":
    my_ip = "192.168.2.99"  # Listen on all available interfaces
    port = 9999
    server(my_ip, port)


[INFO] Server listening on 192.168.2.99:9999
[INFO] Connection received from ('192.168.2.99', 59294)
[INFO] Received message: HELLO
[INFO] Connection received from ('192.168.2.99', 59296)
[INFO] Received message: TONE
[INFO] Playing tone...
[INFO] Activating buzzer at 1000Hz for 0.5s
[INFO] Buzzer OFF
[INFO] Connection received from ('192.168.2.99', 59298)
[INFO] Connection received from ('192.168.2.99', 59300)
[INFO] Received message: HELLO
[INFO] Received message: HELLO
[INFO] Connection received from ('192.168.2.99', 59302)
[INFO] Received message: DISCONNECT
[INFO] Client disconnected.
[INFO] Connection received from ('192.168.2.99', 59304)
[INFO] Connection received from ('192.168.2.99', 59306)
[INFO] Received message: HELLO
[INFO] Received message: HELLO
[INFO] Connection received from ('192.168.2.99', 59308)
[INFO] Received message: TONE
[INFO] Playing tone...
[INFO] Activating buzzer at 1000Hz for 0.5s
[INFO] Buzzer OFF
[INFO] Connection received from ('192.168.2.99', 59310)
[I

In [ ]:
# Define the tone function to generate a square wave
def tone(freq, duration):
    period = 1.0 / freq
    half_period = period / 2
    cycles = int(freq * duration)
    
    for _ in range(cycles):
        write_gpio(0, 1)  # Set GPIO pin high
        time.sleep(half_period)  # Sleep for half the period
        write_gpio(0, 0)  # Set GPIO pin low
        time.sleep(half_period)  # Sleep for half the period

        # Define the button handling function
def server(server_ip, server_port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # ✅ Allow address reuse
    sock.bind((server_ip, server_port))
    sock.listen(5)  # Allow multiple clients
    print(f"[INFO] Server listening on {server_ip}:{server_port}")

    while True:
        connection_socket, client_address = sock.accept()
        multiprocessing.Process(target=handle_client, args=(connection_socket,)).start()

def handle_client(connection_socket):
    try:
        message = connection_socket.recv(2048).decode()
        print("Received message:", message)
        if message == 'TONE':
            tone(1000, 0.5)  # Activate buzzer on receiving board
    finally:
        connection_socket.close()
# Define the client function
def client(server_ip_board2, server_port, action):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((server_ip_board2, server_port))
    print(f"Connected to server {server_ip_board2}:{server_port}")

    sock.sendall(action.encode())  # Send the action ('TONE' or 'DISCONNECT')
    sock.close()
    print("Socket closed")
# Define the button handling function
btns = [base.buttons[i] for i in range(4)]  # Define btns as a list of button objects
def get_btns(server_ip_board2, server_port):
    while True:
        if btns[0].read() != 0:
            print("Connecting to server")
            client(server_ip_board2, server_port, 'HELLO')
        if btns[1].read() != 0:
            print("Activate buzzer on other board")
            client(server_ip_board2, server_port, 'TONE')
        if btns[2].read() != 0:
            print("Disconnecting from server")
            client(server_ip_board2, server_port, 'DISCONNECT')
        if btns[3].read() != 0:
            print("Only buttons 1, 2, and 3 are operational")
        time.sleep(0.1)  # Debounce delay

# Main function to start processes
def main():
    my_ip = '192.168.2.99'  # Replace with actual IP
    other_board_ip = '192.168.2.99'  # Replace with the other board’s IP
    port = 9999
    
# Start server process on this board
    server_process = multiprocessing.Process(target=server, args=(my_ip, port))
    server_process.start()

    try:
        get_btns(other_board_ip, port)  # Client logic runs on this board
    except KeyboardInterrupt:
        print("\n[INFO] KeyboardInterrupt detected. Stopping server...")
        server_process.terminate()  # Stop the server process
        server_process.join()  # Ensure it fully exits
    finally:
        print("[INFO] Cleanup complete. Exiting program.")
if __name__ == "__main__":
    main()
